In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import os
os.chdir(r"C:\Users\meebh\iim materials\job-hackthon\analytics vidya marathon")

In [3]:
train = pd.read_csv("train.csv")

In [4]:
train["Health Indicator"] = train["Health Indicator"].fillna(train["Health Indicator"].mode()[0])

In [5]:
train["Holding_Policy_Duration"] = train["Holding_Policy_Duration"].fillna(train["Holding_Policy_Duration"].mode()[0])

In [6]:
train["Holding_Policy_Type"] = train["Holding_Policy_Type"].fillna(train["Holding_Policy_Type"].mean())

In [7]:
#Ordinal Columns to perform Label Encoding
columns_to_le = ["Accomodation_Type","Reco_Insurance_Type","Health Indicator"]
#Label Encoding the categorical columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in columns_to_le:
    train[col] = le.fit_transform(train[col])

In [8]:
columns_to_oe = ["Is_Spouse"]
#Label Encoding the Nominal categorical columns
from category_encoders import OneHotEncoder
oe = OneHotEncoder(cols = columns_to_oe)
train = oe.fit_transform(train)

In [9]:
train.drop(axis=1,columns=['City_Code'],inplace=True)

In [10]:
train["Holding_Policy_Duration"].replace("14+",15,inplace=True)

In [11]:
X = train.iloc[:,:-1].values
y = train.iloc[:,-1].values

In [42]:
from sklearn.model_selection import train_test_split

#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.5, random_state = 0)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1, 
                                                 random_state = 0, stratify = y)

In [35]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, criterion='entropy',random_state=0)
rf.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', random_state=0)

In [43]:
y_pred = rf.predict(X_test)

In [16]:
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, roc_curve, auc


In [44]:
accuracy_score(y_test,y_pred)

0.7561406956179996

In [45]:
f1_score(y_test,y_pred)

0.02971071149335418

In [46]:
confusion_matrix(y_test,y_pred)

array([[3829,   39],
       [1202,   19]], dtype=int64)

In [47]:
probs = rf.predict_proba(X_test)
prob_positive = probs[:,1]
fpr, tpr, threshold = roc_curve(y_test,prob_positive)
roc_auc = auc(fpr,tpr)
roc_auc

0.6099744686869816

In [48]:
test = pd.read_csv("test.csv")

In [49]:
test["Health Indicator"] = test["Health Indicator"].fillna(test["Health Indicator"].mode()[0])

In [50]:
test["Holding_Policy_Duration"] = test["Holding_Policy_Duration"].fillna(test["Holding_Policy_Duration"].mode()[0])

In [51]:
test["Holding_Policy_Type"] = test["Holding_Policy_Type"].fillna(test["Holding_Policy_Type"].mean())

In [52]:
columns_to_le = ["Accomodation_Type","Reco_Insurance_Type","Health Indicator"]
#Label Encoding the categorical columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in columns_to_le:
    test[col] = le.fit_transform(test[col])

In [53]:
test["Holding_Policy_Duration"].replace("14+",15,inplace=True)

In [54]:
columns_to_oe = ["Is_Spouse"]
#Label Encoding the Nominal categorical columns
from category_encoders import OneHotEncoder
oe = OneHotEncoder(cols = columns_to_oe)
test = oe.fit_transform(test)

In [55]:
test.drop(axis=1,columns=['City_Code'],inplace=True)

In [56]:
y_pred = rf.predict(test)

In [57]:
submission = pd.read_csv('sample_submission.csv')
final_predictions =y_pred
submission['Response'] = final_predictions
#only positive predictions for the target variable
submission['Response'] = submission['Response'].apply(lambda x: 0 if x<0 else x)
submission.to_csv('my_submission3.csv', index=False)